# 0

In [1]:
#其他的库
import numpy as np
import pandas as pd
import scipy as sp 
from sklearn.utils import resample
#封装进度条的函数
from tqdm import tqdm   

#导入自己的函数
import mydef

# 1.0

In [2]:
df=pd.read_csv('./DATA/redshift_bin.csv',index_col=[0,1])
df

REDSHIFT  g_SDSS  gerr_SDSS  r_SDSS  rerr_SDSS  \
red_bin_index org_index                                                   
0             0            0.2235  19.480      0.021  18.707      0.018   
              1            0.2340  19.600      0.027  18.677      0.016   
              2            0.2449  17.669      0.020  17.585      0.014   
              3            0.2509  18.951      0.025  18.656      0.018   
              4            0.2570  18.858      0.017  18.440      0.016   
...                           ...     ...        ...     ...        ...   
51            5161         4.5180  22.504      0.150  20.194      0.034   
              5162         4.5211  22.930      0.180  20.830      0.039   
              5163         4.5863  23.063      0.197  20.140      0.027   
              5164         4.7160  23.962      0.372  21.722      0.075   
              5165         4.8327  24.895      0.508  21.502      0.064   

                         i_SDSS  ierr_SDSS      LOGBH  LOGEDD_RATIO  \
red_bin_index org_index                                               
0             0          18.221      0.013   8.562371     -1.670831   
              1          18.105      0.019   8.468839     -1.452045   
              2          17.438      0.016   7.804190     -0.662985   
              3          18.411      0.021   8.305248     -1.562993   
              4          18.222      0.014   8.119494     -1.139233   
...                         ...        ...        ...           ...   
51            5161       19.296      0.023   8.903947      0.447643   
              5162       19.945      0.031   9.851813     -1.093724   
              5163       19.073      0.018  10.409191     -1.177417   
              5164       20.119      0.035   9.646668     -0.637838   
              5165       20.060      0.034  10.046684     -1.104105   

                           LOGLBOL  ...  rerr_HSC      i_HSC  ierr_HSC  \
red_bin_index org_index             ...                                  
0             0          44.991911  ...  0.000591  18.374371  0.000352   
              1          45.117165  ...  0.000744  18.704962  0.000426   
              2          45.241575  ...  0.000497  18.000725  0.000549   
              3          44.842626  ...  0.000497  18.441834  0.000404   
              4          45.080632  ...  0.000581  18.287029  0.000376   
...                            ...  ...       ...        ...       ...   
51            5161       47.451960  ...  0.001292  19.308536  0.000575   
              5162       46.858459  ...  0.001731  19.759323  0.001391   
              5163       47.332144  ...  0.001490  19.280464  0.000837   
              5164       47.109201  ...  0.004149  20.580292  0.001420   
              5165       47.042949  ...  0.003383  20.393879  0.001125   

                         g_SDSS-HSC  r_SDSS-HSC  i_SDSS-HSC           MBH  \
red_bin_index org_index                                                     
0             0            0.240975   -0.030987   -0.153371  3.650659e+08   
              1           -0.604891   -0.567598   -0.599962  2.943333e+08   
              2           -0.851733   -0.698943   -0.562725  6.370741e+07   
              3           -0.031300   -0.089213   -0.030833  2.019519e+08   
              4            0.058298   -0.126313   -0.065029  1.316722e+08   
...                             ...         ...         ...           ...   
51            5161         0.413973    0.237915   -0.012536  8.015801e+08   
              5162         0.394741    0.299006    0.185677  7.109073e+09   
              5163         0.087036    0.201562   -0.207464  2.565611e+10   
              5164        -0.131718   -0.013556   -0.461292  4.432702e+09   
              5165        -0.273055   -0.429934   -0.333879  1.113484e+10   

                         EDD_RATIO          LBOL  half_g_SDSS+HSC  
red_bin_index org_index                                            
0             0         

# mean&median

In [3]:
df_mean_0=pd.DataFrame()
for i in range(52):
    df_mean_0[i]=df.loc[i].mean()
df_mean = pd.DataFrame(df_mean_0.values.T, index=df_mean_0.columns, columns=df_mean_0.index) 
#SDSS_NAME的数据是string所以数据丢失了，不过没有关系
#ra,dec,RA,DEC,HSCid这些数据也是没有意义的
#对于BH的三个对数数据，直接平均也是没有意义的
df_mean.to_csv('./DATA/redshift_bin_mean.csv')

In [4]:
df_median_0=pd.DataFrame()
for i in range(52):
    df_median_0[i]=df.loc[i].median()
df_median = pd.DataFrame(df_median_0.values.T, index=df_median_0.columns, columns=df_median_0.index) 
#SDSS_NAME的数据是string所以数据丢失了，不过没有关系
#ra,dec,RA,DEC,HSCid这些数据也是没有意义的
df_median.to_csv('./DATA/redshift_bin_median.csv')

# mean bootstrapping

In [5]:
#先将红移bin的数据提取出来.#注释一下，df.loc[i]是第i个红移bin
df_list=[]
for i in range(52):
    df_list.append(df.loc[i])

In [6]:
def My_Bs_Mean(list,n_samples=100,n_loop=1000):


    #这里是样本抽一百次的，抽样重复一千次的
    '''
    对一个样本list操作,bootstrapping过程中，每次抽样抽n_samples个，一共抽n_loop次
    返回bootstrapping的平均值的分布列表（未排序），是一个n_loop长度的list
    '''
    i=n_loop
    n=n_samples
    bs_mean=[];
    for i in range(i):##!!循环次数在这里
        bootstrap_samples = resample(list,n_samples=n ,replace=1)#抽样个数再n——sample里
        bs_mean.append(np.mean(bootstrap_samples))
    return bs_mean


In [7]:
df_sigma=pd.DataFrame(columns=df.columns+'_err')

for i in tqdm(range(52)):
    t_list=[]
    for j in df.columns:
        Sigma= lambda x : np.sqrt(np.var(My_Bs_Mean( x ),ddof=1)) 
        t_list.append(Sigma(df_list[i][j]))
    t_series=pd.Series(t_list,index=df_sigma.columns)   
    df_sigma=df_sigma.append(t_series,ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [08:39<00:00,  9.99s/it]


In [8]:
df_sigma.to_csv('./DATA/redshift_bin_meanerr.csv')

# median bootstrapping


In [9]:
def My_Bs_median(list,n_samples=100,n_loop=1000):


    #这里是样本抽一百次的，抽样重复一千次的
    '''
    对一个样本list操作,bootstrapping过程中，每次抽样抽n_samples个，一共抽n_loop次
    返回bootstrapping的中位值的分布列表（未排序），是一个n_loop长度的list
    '''
    i=n_loop
    n=n_samples
    bs_median=[];
    for i in range(i):##!!循环次数在这里
        bootstrap_samples = resample(list,n_samples=n ,replace=1)#抽样个数再n——sample里
        bs_median.append(np.median(bootstrap_samples))
    return bs_median


In [10]:
df_sigma_median=pd.DataFrame(columns=df.columns+'_err')

for i in tqdm(range(52)):
    t_list=[]
    for j in df.columns:
        sigma_median= lambda x : np.sqrt(np.var(My_Bs_median( x ),ddof=1)) 
        t_list.append(sigma_median(df_list[i][j]))
    t_series=pd.Series(t_list,index=df_sigma_median.columns)   
    df_sigma_median=df_sigma_median.append(t_series,ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [08:19<00:00,  9.61s/it]


In [11]:
df_sigma_median.to_csv('./DATA/redshift_bin_medianerr.csv')

SyntaxError: invalid syntax (<ipython-input-2-8bf6bfcabf46>, line 1)